# Final Project
The goal of the project is to explore a number of datasets that may be associated with political instability in the U.S. The data was taken from the Seshat Databank under Creative Commons Attribution Non-Commercial (CC By-NC SA) licensing. You are asked to follow the steps and/or answer the questions below. It is permitted to work on the project in pairs or individually (1-2 students per project). If working in a group, please ensure that you include the names of both team members in your submission and on the document itself!

### Group Members
- Yordi Hernandez
- Dariel Cruz Rodriguez

***

In [33]:
# Put any modules we need to import here so it all loads at the start of the code.

import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np

## Data Wrangling

* [X] Read the (short) code book.
* [ ] Numerical data need to be uploaded, interpolated, and properly saved. For the purpose
of this project, interpolate each variable such that you obtain one point per year (within
the range of available data).
* [ ] Calculate (and then interpolate) the political instability index.
* [ ] Display the DataFrame with all of the columns and the interpolated data for the years
1901-1910.

In [34]:
# Yordi
# - PltEVI.csv
# - PltHeight.csv
# - PltHSUS.csv

In [ ]:
# Dariel
# - PltPolarizaiton.csv
# - PltWageGDPRatio.csv
# - USPVdatabase.xlsx

pltpolar = pd.read_csv("files/PltPolarization.csv", header=None, names=["year", "polarization"])
pltwage = pd.read_csv("files/PltWageGDPRatio.csv", header=None, names=["year", "ratio"])
uspv = pd.read_csv("files/USPVdatabase.csv")

,year,polarization
0,1790.154061,0.552307
1,1790.120750,0.545883
2,1790.116895,0.540504
3,1790.113039,0.535125
4,1790.391398,0.528719


,year,ratio
0,1790.671138,1.157565
1,1790.533207,1.146055
2,1790.446953,1.134650
3,1790.366525,1.122982
4,1790.222768,1.111003


,code,year,type,subtype,fatalities,location,source,description
0,NaN,1782.05,lynching,political,1,South Carolina,Gilje,May: After Judge Aedewus (??) Burke's Court fa...
1,NaN,1784.03,lynching,political,1,"Montgomery, Ulster Co. NYC",Gilje,March: A Tory was shot when he attempted to se...
2,NaN,1784.07,riot,economic,2,"Ft. Dickinson, Wyoming Valley, PA",Gilje,Pennamite-Yankee War. July: John Franklin and ...
3,NaN,1784.07,riot,economic,2,"Wyoming Valley, PA",Gilje,Pennamite-Yankee War. July: Yankees and Penna...
4,NaN,1784.09,riot,economic,2,"Ft. Dickinson, Wyoming Valley, PA",Gilje,Pennamite-Yankee War. September: In August a ...


In [36]:
# Dariel

#####################################
# Political Polarizaiton (pltpolar) #
#####################################

# Interpolating pltpolar by taking the mean so we have exactly one data point per year
# Structure: year, distance between the average scores of the Democrats/Republicans for each Congress
pltpolar["time"] = np.floor(pltpolar["year"])
pltpolar = pltpolar.groupby("time")["polarization"].mean().reset_index()
pltpolar["time"] = pltpolar["time"].apply(lambda x : int(x))

######################################
# Political Wage/GDP Ratio (pltwage) #
######################################

# Interpolating pltwage by taking the mean so we have exactly one data point per year
# Structure: year, ratio of blue collar wages/gdp per year
pltwage["time"] = np.floor(pltwage["year"])
pltwage = pltwage.groupby("time")["ratio"].mean().reset_index()
pltwage["time"] = pltwage["time"].apply(lambda x : int(x))

######################################
# Political Violence Database (uspv) #
######################################
uspv = uspv.drop(columns=["code","source","description", "location"])
uspv["time"] = np.floor(uspv["year"])
uspv = uspv.dropna()
uspv["time"] = uspv["time"].apply(lambda x : int(x))

pivot_type = pd.crosstab(uspv['time'], uspv['type'])
uspv = pivot_type

######################################
# JOINING DARIEL'S DATASETS TOGETHER #
######################################

# We can merge with Yordi's datasets later on the time column
dariel_joined = pd.merge(pltpolar, pltwage, on="time", how="outer")
dariel_joined = pd.merge(dariel_joined, uspv, on="time", how="outer")
dariel_joined[["assassination", "executions", "insurrection", "lynching",
               "mass suicide", "rampage", "riot", "terrorism", "war"]] = \
dariel_joined[["assassination", "executions", "insurrection", "lynching",
               "mass suicide", "rampage", "riot", "terrorism", "war"]].fillna(0)

years_wanted = [1901, 1902, 1903, 1904, 1905, 1906, 1907, 1908, 1909, 1910]
dariel_joined = dariel_joined[dariel_joined["time"].isin(years_wanted)]
dariel_joined["time"] = dariel_joined["time"].apply(lambda x : str(x))
dariel_joined["ratio"] = dariel_joined["ratio"].interpolate(method='linear')
dariel_joined = dariel_joined.reset_index().drop(columns="index")
dariel_joined

,time,polarization,ratio,assassination,executions,insurrection,lynching,mass suicide,rampage,riot,terrorism,war
0,1901,0.845472,0.651275,1.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,0.0
1,1902,0.860217,0.645180,0.0,0.0,0.0,3.0,0.0,0.0,2.0,0.0,0.0
2,1903,0.868912,0.639086,0.0,0.0,0.0,5.0,0.0,0.0,9.0,0.0,0.0
3,1904,0.877794,0.627711,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
4,1905,0.890111,0.606085,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
5,1906,0.898414,0.596688,0.0,0.0,0.0,3.0,0.0,0.0,10.0,0.0,0.0
6,1907,0.891646,0.607063,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0
7,1908,0.882256,0.610940,0.0,0.0,0.0,9.0,0.0,0.0,5.0,0.0,0.0
8,1909,0.873668,0.608747,0.0,0.0,0.0,7.0,0.0,0.0,1.0,0.0,0.0
9,1910,0.860964,0.602996,0.0,0.0,0.0,5.0,0.0,0.0,19.0,1.0,0.0


## Exploratory Data Analysis
* [ ] Conduct an exploratory data analysis
* [ ] Summarize your main findings (most interesting/insightful conclusions) in a short para-
graph and include appropriate visualizations.

## Find the best regressor
Find the best regressor that would predict the instability index from the various predictors. To be clear, you are asked to compare a limited set of regressors of your choice – not to identify the theoretically optimal one.

* [ ] Explain your modeling choices.
* [ ] Interpret any evaluation metrics you use.
* [ ] Summarize your conclusions in a short paragraph, i.e., the most interesting conclusion(s), the model that produced it, and how the model was chosen. You may include a figure if you find it helpful.

## Find the best dimensionality reduction for regression
You can restrict this part to reducing the data to two dimensions, to three dimensions, or explore both options. You can test your variables using the best regressor found in the previous section or a small number of regressors (2-3 models at most).

* [ ] Explain modeling choices and evaluation metrics.
* [ ] Summarize your conclusions in a short paragraph, i.e., the most interesting conclusion(s), the model that produced it, and how the model was chosen. You may include a figure if you find it helpful

## Find the best dimensionality reduction for unsupervised classification

Use only the predictor columns and not the outcome (instability) for classification. You can restrict this part to reducing the data to two dimensions, to three dimensions, or explore both options.

* [ ] You can test your variables using k-means or a small number of classifiers (2-3 models at most).
* [ ] Explain modeling choices and evaluation metrics.
* [ ] Summarize your conclusions in a short paragraph, i.e., the most interesting conclusion(s), the model that produced it, and how the model was chosen. You may include a figure if you find it helpful.

## Briefly explore the clusters of instability scores

Consider the cluster labels from the best clustering scheme from previous section or from clustering using all/most of the original features. Apply it to the corresponding records of the outcome column (instability).

* [ ] Create a visualization of the results.
* [ ] Summarize your conclusions. To be clear, the summary can be very short, and may be that the clusters do not exhibit any discernable or interpretable pattern. You may include a figure if you find it helpful.

## Consider a real life modeling/prediction problem

* [ ] Try a large number (100? 1000?) different models
* [ ] Examine their performances
* [ ] Select the one that scores best on your performance metric of choice
* [ ] Briefly discuss the potential disadvantage (or potential danger) of such and approach and how you might go about mitigating it.